#Hard coded values
Column names
'createdatl,'updatedatl','marketproduct_id','Eddress ID','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','list_price,'g10_price','transfer_groups','isvisible'

Segment filter
"Asset", "Delist", "None", "Test", "Only Promo"


Cities
"Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen","Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München", "Nürnberg/Fürth","Frankfurt","Offenbach","Stuttgart","Heidelberg","Mannheim","Karlsruhe","Darmstadt","Augsburg"

#Using Instructions
1 - save panel report on desktop and rename until date. (ex - G10andGetirBuyuk_AllProducts_without_TRDE_20230605.xlsx)
2 - save eddress export all on desktop and rename as Inventory.xls
3 - run the code - you will have a sheet on your desktop "Status_update"





In [6]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
today = dt.now()

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE_old.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)

#select only required columns

Assortment = Assortment[['createdatl','updatedatl','Getir Product Id','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','transfer_groups','isvisible']]

#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#------------------------------

#creating dictionary to match the product ids

Path_B = "/Users/swaroopbasavarajmusti/Desktop/Inventory.xls"
Dictionary = pd.read_excel(Path_B)

Dictionary = Dictionary[Dictionary['Getir Product Id'].notna()].reset_index(drop = True)
Dictionary = Dictionary[['Getir Product Id','Product Id','Product SKU','Enabled / Disabled']]


#------------------------------

#left joining Panel assortment with eddress ID dictionary

Assortment_Joined = pd.merge(Assortment, 
                      Dictionary, 
                      on ='Getir Product Id', 
                      how ='left')


#------------------------------

#filtering only required columns
Assortment_Joined = Assortment_Joined[['createdatl','updatedatl','Getir Product Id','Product Id','Product SKU','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','isvisible','transfer_groups']]

#convert datetime format to just date
Assortment_Joined['createdatl'] = Assortment_Joined['createdatl'].dt.date
Assortment_Joined['updatedatl'] = Assortment_Joined['updatedatl'].dt.date


#------------------------------
cities = ["Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen",\
          "Hannover","Düsseldorf","Dortmund","Köln","Essen","München",\
          "NürnbergFürth","Frankfurt","Offenbach","Stuttgart","Heidelberg",\
          "Mannheim","Karlsruhe","Darmstadt","Augsburg","Bonn"]
#------------------------------

#renaming and del column
Assortment_Joined.rename(columns={'createdatl':'Product Card Creation Date'}, inplace=True)
Assortment_Joined.rename(columns={'updatedatl':'Delisting date (TBD)'}, inplace=True)
Assortment_Joined.rename(columns={'net_net_buying_price_with_vat':'Buying Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'g10_price':'Selling Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'isvisible':'Live Getir app'}, inplace=True)
Assortment_Joined.rename(columns={'Product Id':'Eddress Product ID'}, inplace=True)
Assortment_Joined.rename(columns={'product_barcode':'GTIN_Panel'}, inplace=True)
Assortment_Joined.rename(columns={'Product SKU':'GTIN_Eddress'}, inplace=True)
Assortment_Joined.rename(columns={'segment_name':'Segment_Panel'}, inplace=True)

#------------------------------
#additional formatting
Assortment_Joined['Delisting date (TBD)'] = ''
#del Assortment_Joined['transfer_groups']
#Assortment_Joined['Order_Status'] = Assortment_Joined['Order_Status'].fillna('FALSE')
#del Assortment_Joined['Enabled / Disabled']
        
#------------------------------

#sorting and reset index
Assortment_Joined.sort_values(by='Product Card Creation Date', ascending = False ,inplace = True)
Assortment_Joined = Assortment_Joined.reset_index(drop=True)

#-------------------------------

#adding mdm data to dataset to get the product status on mdm

Path_D = "/Users/swaroopbasavarajmusti/Desktop/mdm_data.xlsm"
mdm_data_all = pd.read_excel(Path_D,'Entities')
mdm_data_all = mdm_data_all.rename(columns=mdm_data_all.iloc[0])
mdm_data_all = mdm_data_all.iloc[1:].reset_index(drop = True)
mdm_data = mdm_data_all

#--------------------------------

#Assortment_Joined.rename(columns={'Product Id':'Eddress Product ID'}, inplace=True)

mdm_data_2 = mdm_data_all[mdm_data_all['Eddress Product ID'].notna()].reset_index(drop =True)

#--------------------------------

#merge Panel data to MDM assortment with eddress ids
MDM_TG_sync_format = pd.merge(mdm_data_2,Assortment_Joined, 
                      on ='Eddress Product ID', 
                      how ='left')

MDM_TG_sync_format = MDM_TG_sync_format\
                        [['Action','Type','ID','Name','Buying Categories','GS1 Taxonomy','GIPID',\
                        'Trade Item Identification: GTIN','Colli','Netsuite Vendor ID',\
                        'Eddress Product ID','transfer_groups']]

#Create columns for cities and MFC update

MDM_TG_sync_format["New_Cities"] = np.nan
MDM_TG_sync_format["New_MFC"] = np.nan
MDM_TG_sync_format['New_Cities'] = MDM_TG_sync_format['New_Cities'].astype(str)
MDM_TG_sync_format['New_MFC'] = MDM_TG_sync_format['New_Cities'].astype(str)

for c in cities:
    for ind, row in MDM_TG_sync_format.iterrows():
        
        if pd.isnull(MDM_TG_sync_format.loc[ind, 'transfer_groups']) == True:
            MDM_TG_sync_format.loc[ind, 'New_Cities'] = ""
        
        elif c in MDM_TG_sync_format.loc[ind, 'transfer_groups']:
            if  c in MDM_TG_sync_format.loc[ind, 'New_Cities']:
                pass
            else:
                x = MDM_TG_sync_format.loc[ind, 'New_Cities']
                y = "||"+c
                temp_val = x+y
                MDM_TG_sync_format.loc[ind, 'New_Cities'] = temp_val
        else:
            pass

mdm_text = MDM_TG_sync_format.copy()
        
#create segment dictionary and segment attributes to sync statuses
MFC_City_Dictionary = {\

'Hamburg': ['BILLSTEDT Getir', 'OTTENSEN', 'EIMSBUSH', 'ALTONA NORD Getir', 'ALTONA', 'BERLINER TOR Getir', 'SIEVEKING', 'ROTHENBAUM', 'HOHELUFT Getir', 'BARMBEK NORD Getir', 'BARMBEK SUD Getir', 'MUHLENKAMP Getir'],\
'Berlin': ['WITTENAU Getir', 'OSLOER STR. Getir', 'WEISSENSEE Getir', 'MOABIT Getir', 'PANKOW', 'SAMARITER', 'MITTE', 'TREPTOW', 'NKLLN', 'MARIENDORF Getir', 'FRIEDENAU', 'SBERG', 'STEGLITZ', 'BAYERISCHER PLATZ Getir', 'GRUNEWALD', 'FBERG', 'SHAUSER', 'PBERG', 'CBURG', 'BOXI Getir', 'MERINGPLATZ Getir', 'CHARLIE', 'FHAIN', 'BKIEZ', 'LIGHTHOUSE'],\
'München': ['OLYMPIAPARK Getir', 'PASING Getir', 'HAIDHAUSEN', 'LEHEL', 'LAIM', 'SCHWANTHALER HÖHE Getir', 'SENDLINGEN', 'HARRAS Getir', 'MAXVORSTADT'],
'Dortmund': ['OSTWALL Getir'],\
'Bremen': ['BUNTENTOR', 'FINDORFF'],\
'Dresden': ['BLASEWITZ', 'KONIG'],\
'Hannover': ['ENGEL', 'HILDE'],\
'NürnbergFürth': ['OPERNHAUS Getir', 'FRANKEN'],\
'Leipzig': ['CONNI', 'ZEIGNER', 'KONSTANTIN'],\
'Düsseldorf': ['FLINGERN', 'HERZOG', 'IMMERMANNSTR. Getir', 'DERENDORF'],\
'Fürth': ['STADTPARK'],\
'Frankfurt': ['EUROPA', 'SACHS', 'OSTEND'],\
'Köln': ['MUHLHEIM Getir', 'NIPPES Getir', 'EHRE', 'INNEN', 'SÜLZ', 'NEUMARKT Getir', 'ZOLLSTOCK Getir'],\
'Stuttgart': ['BOEBLINGER', 'ROTEBÜHL'],\
'Heidelberg': ['BERGHEIMER'],\
'Karlsruhe': ['ZIRKEL', 'MUEHLBURG'],\
'Augsburg': ['KAROLIN'],\
'Mannheim': ['QUADRATE'],\
'Offenbach': ['DOM'],\
'Essen': ['HOHENZOLLERN'],\
'Bonn': ['FRIEDRICH'],\
'Darmstadt': ['LUISE']
                       
}

#loop through transfer groups and get city and mfc values

value = None
for c in cities:
    for ind, row in MDM_TG_sync_format.iterrows(): 
        if c in MDM_TG_sync_format.loc[ind, 'New_Cities']:
            value = MFC_City_Dictionary[c]
            x = MDM_TG_sync_format.loc[ind, 'New_MFC']
            y = "||".join(value)
            temp_val = x+"||"+y
            MDM_TG_sync_format.loc[ind, 'New_MFC'] = temp_val 
            
        else:
            pass

#format the values based on mdm requirements
#remove nan
MDM_TG_sync_format["New_Cities"] = MDM_TG_sync_format["New_Cities"].astype(str)
MDM_TG_sync_format["New_Cities"] = MDM_TG_sync_format["New_Cities"].str[5:]

MDM_TG_sync_format["New_MFC"] = MDM_TG_sync_format["New_MFC"].astype(str)
MDM_TG_sync_format["New_MFC"] = MDM_TG_sync_format["New_MFC"].str[5:]

#rename hannover, munchen and nurnbergfurth
MDM_TG_sync_format['New_Cities'] = MDM_TG_sync_format['New_Cities'].str.replace('Hannover','Hanover')
MDM_TG_sync_format['New_Cities'] = MDM_TG_sync_format['New_Cities'].str.replace('München','Munich')
MDM_TG_sync_format['New_Cities'] = MDM_TG_sync_format['New_Cities'].str.replace('NürnbergFürth','Nuremberg')

#bring the format to MDM tempate
MDM_TG_sync_format['New_MFC_2'] = MDM_TG_sync_format['New_MFC']

#delete tradeitems which do not have matching attributs and fill colli as N/A
Absent_TG = MDM_TG_sync_format[~MDM_TG_sync_format['transfer_groups'].notna()]
Present_TG = MDM_TG_sync_format[~MDM_TG_sync_format['transfer_groups'].isna()]

Present_TG["Colli"] = Present_TG["Colli"].fillna("N/A")

MDM_TG_Upload_format = Present_TG[['Action','Type','ID','Name','Buying Categories','GS1 Taxonomy','GIPID',\
                        'Trade Item Identification: GTIN','Colli','Netsuite Vendor ID',\
                        'Eddress Product ID','New_Cities','New_MFC','New_MFC_2']].reset_index(drop= True)

MDM_TG_Upload_format.to_excel("/Users/swaroopbasavarajmusti/Desktop/MDM_TG_Upload_format.xlsx")



/var/folders/67/njs_9h493jg413mvx8b246nr0000gp/T/ipykernel_7795/4287354546.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Present_TG["Colli"] = Present_TG["Colli"].fillna("N/A")
